In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [6]:
ab1 = np.load('/kaggle/input/image-colorization/ab/ab/ab1.npy')[:1000,:,:,:]
# ab2 = np.load('/kaggle/input/image-colorization/ab/ab/ab2.npy')
# ab3 = np.load('/kaggle/input/image-colorization/ab/ab/ab3.npy')

In [7]:
gray_scale = np.load('/kaggle/input/image-colorization/l/gray_scale.npy')[:len(ab1), :, :]
# gray_scale = (np.array(gray_scale[:len(ab1), :, :]))

In [8]:
print(gray_scale.shape)
print(ab1.shape)
# print(ab2.shape)
# print(ab3.shape)

In [9]:
rgb_data = []
for i in range(0,len(ab1)):
    a = cv2.merge((gray_scale[i],ab1[i,:,:,0],ab1[i,:,:,1]))
    c = (cv2.cvtColor(a, cv2.COLOR_LAB2RGB))
    rgb_data.append(c.T.astype('float'))
# lab1 = len(ab1)
# for i in range(0, len(ab2)):
#     a = cv2.merge((gray_scale[i + lab1],ab2[i,:,:,0],ab2[i,:,:,1]))
#     c = cv2.cvtColor(a, cv2.COLOR_LAB2RGB)
#     rgb_data.append(c.T)
# lab12 = len(ab1)+len(ab2)
# for i in range(0, len(ab3)):
#     a = cv2.merge((gray_scale[i + lab12],ab3[i,:,:,0],ab3[i,:,:,1]))
#     c = cv2.cvtColor(a, cv2.COLOR_LAB2RGB)
#     rgb_data.append(c.T)
del(ab1)

In [10]:
rgb_data = (np.array(rgb_data)-128)/255
gray_scale = gray_scale.astype('float')
gray_scale = (np.array(gray_scale[:len(rgb_data), :, :])- 128)/255

In [11]:
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import datasets, transforms
import os, shutil, time

In [12]:
use_gpu = torch.cuda.is_available()
print(use_gpu)

In [13]:
class ColorizationNet(nn.Module):
  def __init__(self, input_size=224):
    super(ColorizationNet, self).__init__()
    MIDLEVEL_FEATURE_SIZE = 128

    ## First half: ResNet
    resnet = models.resnet18(num_classes=365) 
    # Change first conv layer to accept single-channel (grayscale) input
    resnet.conv1.weight = nn.Parameter(resnet.conv1.weight.sum(dim=1).unsqueeze(1)) 
    # Extract midlevel features from ResNet-gray
    self.midlevel_resnet = nn.Sequential(*list(resnet.children())[0:6])

    ## Second half: Upsampling
    self.upsample = nn.Sequential(     
      nn.Conv2d(MIDLEVEL_FEATURE_SIZE, 128, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(128),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(64),
      nn.ReLU(),
      nn.Upsample(scale_factor=2),
      nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1),
      nn.Upsample(scale_factor=2)
    )

  def forward(self, input):

    # Pass input through ResNet-gray to extract features
    midlevel_features = self.midlevel_resnet(input)

    # Upsample to get colors
    output = self.upsample(midlevel_features)
    return output

In [14]:
model = ColorizationNet()



In [15]:
criterion = nn.MSELoss()

In [16]:
if use_gpu:
    criterion = criterion.cuda()
    model = model.cuda()
    

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.0)

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

gray_scale = np.reshape(gray_scale, (len(gray_scale), 1, 224,224))
input_img = torch.from_numpy(gray_scale).float().to(device)
target = torch.from_numpy(rgb_data).float().to(device)

In [19]:
print(input_img.shape)
print(target.shape)

In [20]:
batch_size = 64
def train():
    losses = []
    for i in range(1, int(len(rgb_data)/batch_size) - 1):
        out = model(input_img[batch_size*(i-1):i*batch_size, :,:,:])
        loss = criterion(out, target[batch_size*(i-1):i*batch_size,:,:,:])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        
    return np.average(losses)
        

In [56]:
for i in range(0, 5000):
    if i % 10 == 0:
        print('Loss in epoch '+ str(i) + ' '+str(train()))

In [51]:
i = 600
test_img = (rgb_data[i,:,:,:]*255 ) + 128
plt.imshow(test_img.T.astype('int'))

In [52]:
plt.imshow(gray_scale[i,0,:,:], cmap = 'gray')

In [55]:
ot = model(torch.from_numpy(np.reshape(gray_scale[i,0,:,:],(1,1,224,224))).float().to(device))
plt.imshow(((ot.cpu().detach().numpy()[0].T)*255 + 128).astype('int'))